In [1]:
import os
import pickle

#Import Data 
path = './data/split_data/'
directory = os.listdir(path)
for file in directory:
    if file.endswith(".pickle"):
        with open(path+file, 'rb') as f:
            placeholder = file.split('.')[0] #Placeholder to modify file name to variable name
            vars()[placeholder] = pickle.load(f)
            print('Loaded', file)

Loaded X_resampled_down.pickle
Loaded X_resampled_up.pickle
Loaded X_test.pickle
Loaded X_train.pickle
Loaded y_resampled_down.pickle
Loaded y_resampled_up.pickle
Loaded y_test.pickle
Loaded y_train.pickle


In [2]:
#Working Copy Figure out how to fit into workbook
# #Import Data From Director and Assign File Name to Variable Name
# def import_data(directory_path, file_extension =".pickle"):
#     data_list = []
#     directory = os.listdir(path)
#     for file in directory:
#         if file.endswith(file_extension):
#             with open(path+file, 'rb') as f:
#                 placeholder = file.split('.')[0] #Placeholder to modify file name to variable name
#                 vars()[placeholder] = pickle.load(f)
#                 data_list.append(vars()[placeholder])
#                 print('Loaded', file)
#     return data_list

# path = './data/split_data/'
# split_data = import_data(path, file_extension =".pickle")

In [3]:
list_data = [X_train,y_train,X_resampled_up, y_resampled_up, X_resampled_down,y_resampled_down]

In [5]:
#Import required pacakages
#Imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN

#Matplotlib
import matplotlib.pyplot as plt
from matplotlib import pyplot
%matplotlib inline

# make prettier plots
%config InlineBackend.figure_format = 'svg' 

#Numpy
import numpy as np

#Pandas
import pandas as pd
import pandas_profiling

#Seaborn
import seaborn as sns
sns.set()

#Sklearn
from sklearn import preprocessing
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import quantile_transform
from sklearn.metrics import median_absolute_error
from sklearn.utils import resample
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import (accuracy_score,
                             f1_score,
                             roc_auc_score,
                             roc_curve,
                             confusion_matrix)

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import (RandomForestClassifier,
                              RandomForestClassifier,
                              RandomForestRegressor,
                              GradientBoostingClassifier, 
                              ExtraTreesClassifier, #For each feature split rule is random, not optimal
                              VotingClassifier, 
                              AdaBoostClassifier, 
                              BaggingRegressor)

from sklearn.model_selection import (cross_val_score,
                                     GridSearchCV,
                                     RandomizedSearchCV,
                                     learning_curve,
                                     validation_curve,
                                     train_test_split,
                                     cross_validate)
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
#XGBoost
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

The methodology that I’ll follow when building the classifiers goes as follows:

Build a pipeline that handles all the steps when fitting the classifier using scikit-learn’s `make_pipeline` which will have two steps:

I. Standardizing the data to speed up convergence and make all features on the same scale.

II. The classifier (estimator) we want to use to fit the model.

2. Use GridSearchCV to tune hyperparameters using 10-folds cross validation. We can use RandomizedSearchCV which is faster and may outperform GridSearchCV especially if we have more than two hyperparameters and the range for each one is very big; however, GridSearchCV will work just fine since we have only two hyperparameters and descent range.

3. Fit the model using training data.

4. Plot both confusion matrix and ROC curve for the best estimator using test data.

Repeat the above steps for `Random Forest`, `Gradient Boosting Trees`, `K-Nearest Neighbors`, `Logistic Regression` and `Support Vector Machine`. Next, pick the classifier that has the highest cross validation f1 score.

# Model 1 - Random Forest
First, fit a random forest classifier using `unsampled`, `upsampled`, and `downsampled` data. Second, evaluate each method using cross validation CV f1-score and pick the one with the highest CV F1 score. The best method will be used to fit the rest of the classifiers

Hyperparameters to tune:

`max_features`: How many features to consider randomly on each split to help avoid having few strong features to be picked on each split and let other features have the chance to contribute. The predictions will be less correlated and the variance of each tree will decrease.

`min_samples_leaf`: How many examples to have for each split to be a final leaf node.

In [6]:
'''
Decision Trees are prone to high variance. Comes at cost of overfitting the training set.

Huristic: Greedy Search
Informatino Entropy
Split wiht entrop minimization rule
Boostrap Aggregating - Sample with Replacement

Ensemble Methods Random Forests
Introduce randomness when building each tree.
For a split, do not take the “best” feature split.
First, randomly choose sqrt(n_feat) features.
Only choose the best split among these.

Bagging - Reduces Bagging
-Average Voting
-Max Voting
looking at bootstrapped samples and running several different models on the data allows the classifier to 
find the best aggregate model that will generalize best to our data.

'''

'\nDecision Trees are prone to high variance. Comes at cost of overfitting the training set.\n\nHuristic: Greedy Search\nInformatino Entropy\nSplit wiht entrop minimization rule\nBoostrap Aggregating - Sample with Replacement\n\nEnsemble Methods Random Forests\nIntroduce randomness when building each tree.\nFor a split, do not take the “best” feature split.\nFirst, randomly choose sqrt(n_feat) features.\nOnly choose the best split among these.\n\nBagging - Reduces Bagging\n-Average Voting\n-Max Voting\nlooking at bootstrapped samples and running several different models on the data allows the classifier to \nfind the best aggregate model that will generalize best to our data.\n\n'

In [ ]:
def bias_metric(y_true, y_pred):
    return np.mean((np.mean(y_pred) - y_true)**2)

def variance_metric(y_true, y_pred):
    return np.var(y_true - y_pred)

In [ ]:
i=0

placeholder_variance=[]
placeholder_bias=[]

variance = []
bias = []
test_range = np.arange(1, 100, 1) #Number of Trees

while i < len(list_data)-1:
    X=list_data[i]
    y=list_data[i+1]
    for r in test_range:
        cv_out = cross_validate(
            estimator=BaggingRegressor(
                DecisionTreeRegressor(random_state=123), n_estimators=r),
            X=X,
            y=y,
            cv=3,
            return_train_score=True,
            scoring={
                "variance": make_scorer(variance_metric),
                "bias": make_scorer(bias_metric)
            },
            n_jobs=-1,
        )
        placeholder_variance.append(np.mean(cv_out['test_variance']))
        placeholder_bias.append(np.mean(cv_out['test_bias']))
        
    variance.append(placeholder_variance)
    bias.append(placeholder_bias)
    placeholder_variance=[] #clear placeholder
    placeholder_bias=[] #clear placeholder
    i+=2

In [ ]:
titles = ['Orignial Data','Smoted','Down-Sampled Training Data','Up-Sampled Training Data', 'Down-Resampled Training Data', 'Up-Resampled Training Data']
for i in range(len(variance)):
    plt.figure(figsize=(8,8),dpi=600)
    plt.plot(test_range, variance[i], label='variance')
    #plt.ylim((0, 0.65))
    plt.xlabel('Number of Estimators')
    plt.ylabel('Variance Score');
    plt.title('Variance Score vs No. of Estimators - ' +titles[i]);

    plt.figure(dpi=150)
    plt.plot(test_range, bias[i], label='bias', c='r')
    #plt.ylim((0, 0.25))
    plt.xlabel('Number of Estimators')
    plt.ylabel('Bias Score');
    plt.title('Bias Score vs No. of Estimators - ' +titles[i]);

# Check Balance of Classes
- Look at proportion of classes to see if we're dealing with balanced or imbalanced data, since each one has its own set of tools to be used when fitting classifiers

# Get number of Yes/No Attritions

In [ ]:
with open('./y_test_smote_set.pickle', 'rb') as file:
        y_test_smote_set = pickle.load(file)
        
with open('./x_test_smote_set.pickle', 'rb') as file:
        x_test_smote_set = pickle.load(file)

In [ ]:
# Build random forest classifier
def random_forest_classifier(X_train, y_train):
    methods_data = {"Original": (X_train, y_train)}
                    #"SMOTE": (X_smoted, y_smoted),
                    #"Up-Sampled": (X_train_up, y_train_up),
                    #"Down-Sampled": (X_train_down, y_train_down),
                    #"Up-Resampled": (X_resampled_up, y_resampled_up),
                    #"Down-Resampled": (X_resampled_down,y_resampled_down)}

    for method in methods_data.keys():
        pip_rf = make_pipeline(StandardScaler(),
                               RandomForestClassifier(n_estimators=500, #Typically Sufficient
                                                      class_weight="balanced",
                                                      random_state=123))

        hyperparam_grid = {
            "randomforestclassifier__n_estimators": [10, 50, 100, 200, 500],
            "randomforestclassifier__max_features": ["sqrt", "log2", 0.4, 0.5], #sqrt/log2 => max_features = sqrt/log2(n_features), 
            "randomforestclassifier__min_samples_leaf": [1, 3, 5, 10, 20, 30],
            "randomforestclassifier__criterion": ["gini", "entropy"]}
        
        gs_rf = GridSearchCV(pip_rf,
                             hyperparam_grid,
                             scoring="precision", #The F1 score can be interpreted as a weighted average of the precision and recall, 
                             #where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of 
                             #precision and recall to the F1 score are equal. The formula for the F1 score is:
                             cv=10, # Number of K-fold cross validation
                             n_jobs=-1) # Setting n_jobs to -1, specifys that all CPUs are used
        
        # training the model on oversampled 4 folds of training set
        gs_rf.fit(methods_data[method][0], methods_data[method][1])

        print(f"\033[1m\033[0mThe best hyperparameters for {method} data:")
        for hyperparam in gs_rf.best_params_.keys():
            print(hyperparam[hyperparam.find("__") + 2:], ": ", gs_rf.best_params_[hyperparam])

        print(f"\033[1m\033[94mBest 10-folds CV precision-score: {gs_rf.best_score_ * 100:.2f}%.")

In [ ]:
random_forest_classifier(X_train, y_train)
# i=0
# import math
# while i < len(list_data)-1:
#     print(titles[math.floor(i/2)])
#     X=list_data[i]
#     y=list_data[i+1]
#     random_forest_classifier(X, y)
#     i+=2

Best Hyperparameters as discovered by RandomForestClassifiera with Best 10-fold CV F1-Score of 98.80%:

- criterion :  gini
- max_features :  sqrt
- min_samples_leaf :  1
- n_estimators :  200
- Best 10-folds CV f1-score: 98.80%.

The upsampled data will be used to fit the rest of the classifiers.

In [ ]:
def Random_Forest_Classifier(X_train, y_train,n_estimators,criterion,max_features,min_samples_leaf):
    # Reassign original training data to upsampled data
    X_train_re, y_train_re = np.copy(X_train), np.copy(y_train)

    # Refit RF classifier using best params
    clf_rf = make_pipeline(StandardScaler(),
                           RandomForestClassifier(n_estimators=n_estimators,
                                                  criterion=criterion,
                                                  max_features=max_features,
                                                  min_samples_leaf=min_samples_leaf,
                                                  class_weight="balanced",
                                                  n_jobs=-1,
                                                  random_state=123))
    clf_rf.fit(X_train_re, y_train_re)
        
    return clf_rf

# Check Balance of Classes
- Look at proportion of classes to see if we're dealing with balanced or imbalanced data, since each one has its own set of tools to be used when fitting classifiers

# Get number of Yes/No Attritions

In [ ]:
#Import validation and test data
# path = './data/split_data/val_test/'
# directory = os.listdir(path)
# for file in directory:
#     if file.endswith(".pickle"):
#         with open(path+file, 'rb') as f:
#             placeholder = file.split('.')[0] #Placeholder to modify file name to variable name
#             vars()[placeholder] = pickle.load(f)
#             print('Loaded', file)

In [ ]:
#val_data_list = [X_val,y_val]
#test_data_list = [X_test,y_test]
test_data_list = [x_test_smote_set,y_test_smote_set]

In [ ]:
n_estimators = 300
criterion = 'gini'
max_features = 'log2'
min_samples_leaf = 1

#Will use up-sampled data per random forest results
X_train = x_train_smoted
y_train = y_train_smoted

#Run RandomForest Classifier per Data
clf_RandomForest = Random_Forest_Classifier(X_train, y_train, n_estimators,criterion,max_features,min_samples_leaf)

#Save Model
#filename = './data/fit_models/random_forest/Up_Sampled_Data_RandomForest.sav'
#pickle.dump(clf_RandomForest, open(filename, 'wb'))

#Predict Y using Fitted Models 
y_pred = clf_RandomForest.predict(test_data_list[0])
#filename = './data/fit_models/random_forest/ypred_Up_Sampled_Data_RandomForest.sav'
#pickle.dump(y_pred, open(filename, 'wb'))

# new_list_data_per_tunining = [X_smoted,y_smoted,X_adasyn,y_adasyn,X_train_up,y_train_up,X_resampled_up,y_resampled_up]
# titles = ['SMOTE','ADASYN','TrainUp','ResampledUp']
# i=0
# while i < len(new_list_data_per_tunining)-1:
#     X=new_list_data_per_tunining[i]
#     y=new_list_data_per_tunining[i+1]
#     clf_RandomForest = Random_Forest_Classifier(X, y, n_estimators,criterion,max_features,min_samples_leaf)
#     #Save Model
#     filename = './data/fit_models/'+titles[math.floor(i/2)]+'.sav'
#     pickle.dump(clf_RandomForest, open(filename, 'wb'))
    
#     #Predict Y using Fitted Models 
#     y_pred = clf_RandomForest.predict(val_data_list[0])
#     filename = './data/fit_models/y_pred/ypred_'+titles[math.floor(i/2)]+'.sav'
#     pickle.dump(y_pred, open(filename, 'wb'))
#     i+=2
# print('Fitting Models Completed')

In [ ]:
#Import y_pred


#path = './data/fit_models/y_pred/'
# directory = os.listdir(path)
# y_pred_list= []
# for file in directory:
#     if file.endswith(".sav"):
#         with open(path+file, 'rb') as f:
#             placeholder = file.split('.')[0] #Placeholder to modify file name to variable name
#             vars()[placeholder] = pickle.load(f)
#             y_pred_list.append(vars()[placeholder])
#             print('Loaded', file)

# Check Balance of Classes
- Look at proportion of classes to see if we're dealing with balanced or imbalanced data, since each one has its own set of tools to be used when fitting classifiers

# Get number of Yes/No Attritions

Let’s refit the Random Forest with Upsampled data using best hyperparameters tuned above and plot confusion matrix and ROC curve using test data.

In [ ]:
def plot_confusion_matrix(y_true, y_pred,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Greens):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    fig, ax = plt.subplots(figsize=(8, 8))
    plt.figure(dpi=600)
    
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           ylabel='True label',
           xlabel='Predicted label')
    ax.set_title(title)

    # Rotate the tick labels and set their alignment.
    #plt.setp(ax.get_xticklabels(), rotation=0, ha="right",rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black",
                    fontsize=20)
    fig.tight_layout()
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
              ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(16)
    ax.grid(None)
    ax.yaxis.set_label_coords(-0.15,0.5)
    ax.xaxis.set_label_coords(0.5,-0.15)
    return ax

# Random Forest Confusion Matrix

In [ ]:
#Print Normalized Confusion Matrix
cm_gb = plot_confusion_matrix(y_test_smote_set, y_pred, title='IBM Attrition Data Random Forest Results Confusion Matrix');
print('Random Forest Classifcation Report: \n ', classification_report(y_test_smote_set,y_pred));

In [ ]:
# # Plot non-normalized confusion matrix
# titles = ['SMOTE','ADASYN','TrainUp','ResampledUp']
# for idx, y_pred in enumerate(y_pred_list):
#     cm = plot_confusion_matrix(y_val, y_pred, title='Confusion Matrix '+titles[idx]+' without normalization');
#     print('Classifcation Report for: \n'+titles[idx], classification_report(y_val,y_pred));

In [ ]:
# path = './data/fit_models/'
# directory = os.listdir(path)
# y_pred_list= []
# for file in directory:
#     if file.endswith(".sav"):
#         with open(path+file, 'rb') as f:
#             placeholder = file.split('.')[0] #Placeholder to modify file name to variable name
#             vars()[placeholder] = pickle.load(f)
#             y_pred_list.append(vars()[placeholder])
#             print('Loaded', file)

In [ ]:
# fitted_models = [SMOTE,ADASYN,TrainUp,ResampledUp]
fitted_models = [clf_RandomForest]

In [ ]:
# titles = ['SMOTE','ADASYN','TrainUp','ResampledUp']
# for idx, model in enumerate(fitted_models):
#     fpr, tpr, thresholds = roc_curve(y_val, model.predict_proba(X_val)[:,1])
#     plt.plot(fpr, tpr,lw=2)
#     plt.plot([0,1],[0,1],c='violet',ls='--')
#     plt.xlim([-0.05,1.05])
#     plt.ylim([-0.05,1.05])

#     plt.xlabel('False positive rate')
#     plt.ylabel('True positive rate')
#     plt.title('Random Forest ROC curve for IBM Employee Retention');
#     print("Random Forest "+titles[idx]+" ROC AUC score = ", roc_auc_score(y_val, model.predict_proba(X_val)[:,1]))

In [ ]:
#Generate ROC Curve for Random Forest
fpr, tpr, thresholds = roc_curve(y_test_smote_set,clf_RandomForest.predict_proba(x_test_smote_set)[:,1])

fig, ax = plt.subplots(figsize=(10, 8))
plt.figure(dpi=600)

ax.plot(fpr, tpr,lw=2)
ax.plot([0,1],[0,1],c='violet',ls='--')
ax.set_xlim([-0.05,1.05])
ax.set_ylim([-0.05,1.05])

ax.set_xlabel('False Positive Rate', fontsize=16)
ax.set_ylabel('True Positive Rate', fontsize=16)
ax.set_title('Random Forest ROC curve for IBM Employee Retention',fontsize=16)

for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
              ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(16)

ax.yaxis.set_label_coords(-0.1,0.5)
ax.xaxis.set_label_coords(0.5,-0.1)
print(f"\033[1m\033[94mRandom Forest Up-Sampled Data ROC AUC score = ", roc_auc_score(y_test_smote_set, clf_RandomForest.predict_proba(x_test_smote_set)[:,1]))


# *NOTE* PAST THIS POINT NOTHING IS AUTOMATED

# Gradient Boosting Trees

Gradient Boosting trees are the same as Random Forest except for:

It starts with small tree and start learning from grown trees by taking into account the residual of grown trees.
More trees can lead to overfitting; opposite to Random Forest.
The two other hyperparameters than max_features and n_estimators that we're going to tune are:

learning_rate: rate the tree learns, the slower the better.
max_depth: number of split each time a tree is growing which limits the number of nodes in each tree.
Let's fit GB classifier and plot confusion matrix and ROC curve using test data.

In [ ]:
# Build Gradient Boosting classifier
pip_gb = make_pipeline(StandardScaler(),
                       GradientBoostingClassifier(loss="deviance",
                                                  random_state=123))

hyperparam_grid = {"gradientboostingclassifier__max_features": ["log2", 0.5],
                   "gradientboostingclassifier__n_estimators": [100, 300, 500],
                   "gradientboostingclassifier__learning_rate": [0.001, 0.01, 0.1],
                   "gradientboostingclassifier__max_depth": [1, 2, 3]}

gs_gb = GridSearchCV(pip_gb,
                      param_grid=hyperparam_grid,
                      scoring= "precision",
                      cv=10,
                      n_jobs=-1)

gs_gb = gs_gb.fit(X_train, y_train)

print("\033[1m" + "\033[0m" + "The best hyperparameters:")
print("-" * 25)
for hyperparam in gs_gb.best_params_.keys():
    print(hyperparam[hyperparam.find("__") + 2:], ": ", gs_gb.best_params_[hyperparam])

print("\033[1m" + "\033[94m" + "Best 10-folds CV f1-score: {:.2f}%.".format((gs_gb.best_score_) * 100))

#Save Model
filename = './data/fit_models/gradient_boosting/gradient_boost_trees_model.sav'
pickle.dump(gs_gb, open(filename, 'wb'))

#Predict Y using Model
y_pred = gs_gb.predict(test_data_list[0])

filename = './data/fit_models/gradient_boosting/gradient_boost_trees_ypred.sav'
pickle.dump(y_pred, open(filename, 'wb'))

In [ ]:
#Print Normalized Confusion Matrix
cm_gb = plot_confusion_matrix(y_test, y_pred, title='IBM Attrition Data Gradient Boosting Forest Results Confusion Matrix');
print('Gradient Boosting Tree Classifcation Report: \n ', classification_report(y_test,y_pred));

In [ ]:
#Generate ROC Curve for Gradient Boosting Tree
fpr, tpr, thresholds = roc_curve(y_test,gs_gb.predict_proba(X_test)[:,1])

fig, ax = plt.subplots(figsize=(10, 8))
plt.figure(dpi=600)

ax.plot(fpr, tpr,lw=2)
ax.plot([0,1],[0,1],c='violet',ls='--')
ax.set_xlim([-0.05,1.05])
ax.set_ylim([-0.05,1.05])

ax.set_xlabel('False Positive Rate', fontsize=16)
ax.set_ylabel('True Positive Rate', fontsize=16)
ax.set_title('Gradient Boosting Trees ROC curve for IBM Employee Retention',fontsize=16)

for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
              ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(16)

ax.yaxis.set_label_coords(-0.1,0.5)
ax.xaxis.set_label_coords(0.5,-0.1)
print(f"\033[1m\033[94mGradient Boosting Up-Sampled Data ROC AUC score = ", roc_auc_score(y_test, gs_gb.predict_proba(X_test)[:,1]))

# K-Nearest Neighbors

KNN is called a lazy learning algorithm because it doesn't learn or fit any parameter. It takes n_neighbors points from the training data closest to the point we're interested to predict it's class and take the mode (majority vote) of the classes for the neighboring point as its class. The two hyperparameters we're going to tune are:

- n_neighbors: number of neighbors to use in prediction.
- weights: how much weight to assign neighbors based on:
  
  "uniform": all neighboring points have the same weight.
  
  "distance": use the inverse of euclidean distance of each neighboring point used in prediction.

Let's fit KNN classifier and plot confusion matrix and ROC curve.



In [ ]:
# Build KNN classifier
pip_knn = make_pipeline(StandardScaler(), KNeighborsClassifier())
hyperparam_range = range(1, 20)

gs_knn = GridSearchCV(pip_knn,
                      param_grid={"kneighborsclassifier__n_neighbors": hyperparam_range,
                                  "kneighborsclassifier__weights": ["uniform", "distance"]},
                      scoring="f1",
                      cv=10,
                      n_jobs=-1)

gs_knn = gs_knn.fit(X_train, y_train)

print("\033[1m" + "\033[0m" + "The best hyperparameters:")
print("-" * 25)
for hyperparam in gs_knn.best_params_.keys():
    print(hyperparam[hyperparam.find("__") + 2:], ": ", gs_knn.best_params_[hyperparam])

print("\033[1m" + "\033[94m" + "Best 10-folds CV f1-score: {:.2f}%.".format((gs_knn.best_score_) * 100))

#Save Model
filename = './data/fit_models/KNN/KNN_model.sav'
pickle.dump(gs_knn, open(filename, 'wb'))

#Predict Y using Model
y_pred = gs_knn.predict(test_data_list[0])

filename = './data/fit_models/KNN/KNN_ypred.sav'
pickle.dump(y_pred, open(filename, 'wb'))

In [ ]:
#Print Normalized Confusion Matrix
cm_gb = plot_confusion_matrix(y_test, y_pred, title='IBM Attrition Data KNN Results Confusion Matrix');
print('KNN Classifcation Report: \n ', classification_report(y_test,y_pred));

In [ ]:
#Generate ROC Curve for KNN
fpr, tpr, thresholds = roc_curve(y_test,gs_knn.predict_proba(X_test)[:,1])

fig, ax = plt.subplots(figsize=(10, 8))
plt.figure(dpi=600)

ax.plot(fpr, tpr,lw=2)
ax.plot([0,1],[0,1],c='violet',ls='--')
ax.set_xlim([-0.05,1.05])
ax.set_ylim([-0.05,1.05])

ax.set_xlabel('False Positive Rate', fontsize=16)
ax.set_ylabel('True Positive Rate', fontsize=16)
ax.set_title('KNN ROC curve for IBM Employee Retention',fontsize=16)

for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
              ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(16)

ax.yaxis.set_label_coords(-0.1,0.5)
ax.xaxis.set_label_coords(0.5,-0.1)
print(f"\033[1m\033[94mKNN Up-Sampled Data ROC AUC score = ", roc_auc_score(y_test, gs_knn.predict_proba(X_test)[:,1]))

# Logistic Regression

For logistic regression, we'll tune three hyperparameters:
- penalty: type of regularization, L2 or L1 regularization.
- C: the opposite of regularization of parameter λ. The higher C the less regularization. We'll use values that cover the full  range between unregularized to fully regularized where model is the mode of the examples' label.
- fit_intercept: whether to include intercept or not.

We won't use any non-linearities such as polynomial features.

In [ ]:
# Build logistic model classifier
pip_logmod = make_pipeline(StandardScaler(),
                           LogisticRegression(class_weight="balanced"))

hyperparam_range = np.arange(0.5, 20.1, 0.5)

hyperparam_grid = {"logisticregression__penalty": ["l1", "l2"],
                   "logisticregression__C":  hyperparam_range,
                   "logisticregression__fit_intercept": [True, False]
                  }

gs_logmodel = GridSearchCV(pip_logmod,
                           hyperparam_grid,
                           scoring="accuracy",
                           cv=2,
                           n_jobs=-1)

gs_logmodel = gs_logmodel.fit(X_train, y_train)

print("\033[1m" + "\033[0m" + "The best hyperparameters:")
print("-" * 25)
for hyperparam in gs_logmodel.best_params_.keys():
    print(hyperparam[hyperparam.find("__") + 2:], ": ", gs_logmodel.best_params_[hyperparam])

print("\033[1m" + "\033[94m" + "Best 10-folds CV f1-score: {:.2f}%.".format((gs_logmodel.best_score_) * 100))

#Save Model
filename = './data/fit_models/logistic_regression/logistic_regression_model.sav'
pickle.dump(gs_logmodel, open(filename, 'wb'))

#Predict Y using Model
y_pred = gs_logmodel.predict(test_data_list[0])

filename = './data/fit_models/logistic_regression/logistic_regression_ypred.sav'
pickle.dump(y_pred, open(filename, 'wb'))

In [ ]:
#Print Confusion Matrix
#Print Normalized Confusion Matrix
cm_gb = plot_confusion_matrix(y_test, y_pred, title='IBM Attrition Data Logistic Regression Results Confusion Matrix');
print('Logistic Regression Classifcation Report: \n ', classification_report(y_test,y_pred));

In [ ]:
#Generate ROC Curve for Logistic Regression
fpr, tpr, thresholds = roc_curve(y_test,gs_logmodel.predict_proba(X_test)[:,1])

fig, ax = plt.subplots(figsize=(10, 8))
plt.figure(dpi=600)

ax.plot(fpr, tpr,lw=2)
ax.plot([0,1],[0,1],c='violet',ls='--')
ax.set_xlim([-0.05,1.05])
ax.set_ylim([-0.05,1.05])

ax.set_xlabel('False Positive Rate', fontsize=16)
ax.set_ylabel('True Positive Rate', fontsize=16)
ax.set_title('Logistic Regression ROC curve for IBM Employee Retention',fontsize=16)

for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
              ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(16)

ax.yaxis.set_label_coords(-0.1,0.5)
ax.xaxis.set_label_coords(0.5,-0.1)
print(f"\033[1m\033[94mLogistic Regression Up-Sampled Data ROC AUC score = ", roc_auc_score(y_test, gs_logmodel.predict_proba(X_test)[:,1]))


# Support Vector Machine (SVM)

SVM is comutationally very expensive to tune it's hyperparameters for two reasons:
- With big datasets, it becomes very slow.
- It has good number of hyperparameters to tune that takes very long time to tune on a CPU.

Therefore, we'll use recommended hyperparameters' values from the paper we mentioned before that showed to yield the best performane on Penn Machine Learning Benchmark 165 datasets. The hyperparameters that we usually look to tune are:

C, gamma, kernel, degree and coef0

In [ ]:
clf_svc = make_pipeline(StandardScaler(),
                        SVC(C=0.01,
                            gamma=0.1,
                            kernel="poly",
                            degree=5,
                            coef0=10,
                            probability=True))

clf_svc = clf_svc.fit(X_train, y_train)

svc_cv_scores = cross_val_score(clf_svc,
                                X=X_smoted,
                                y=y_smoted,
                                scoring="f1",
                                cv=10,
                                n_jobs=-1)

# Print CV
print("\033[1m" + "\033[94m" + "The 10-folds CV f1-score is: {:.2f}%".format(
       np.mean(svc_cv_scores) * 100))

#Save Model
filename = './data/fit_models/SVC/SVC_model.sav'
pickle.dump(clf_svc, open(filename, 'wb'))

#Predict Y using Model
y_pred = clf_svc.predict(test_data_list[0])

filename = './data/fit_models/SVC/SVC_ypred.sav'
pickle.dump(y_pred, open(filename, 'wb'))

In [ ]:
#Print Confusion Matrix
#Print Normalized Confusion Matrix
cm_gb = plot_confusion_matrix(y_test, y_pred, title='IBM Attrition Data SVM Results Confusion Matrix');
print('SVM Classifcation Report: \n ', classification_report(y_test,y_pred));

In [ ]:
#Generate ROC Curve for SVC
fpr, tpr, thresholds = roc_curve(y_test,clf_svc.predict_proba(X_test)[:,1])

fig, ax = plt.subplots(figsize=(10, 8))
plt.figure(dpi=600)

ax.plot(fpr, tpr,lw=2)
ax.plot([0,1],[0,1],c='violet',ls='--')
ax.set_xlim([-0.05,1.05])
ax.set_ylim([-0.05,1.05])

ax.set_xlabel('False Positive Rate', fontsize=16)
ax.set_ylabel('True Positive Rate', fontsize=16)
ax.set_title('SVC ROC curve for IBM Employee Retention',fontsize=16)

for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
              ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(16)

ax.yaxis.set_label_coords(-0.1,0.5)
ax.xaxis.set_label_coords(0.5,-0.1)
print(f"\033[1m\033[94mSVC Up-Sampled Data ROC AUC score = ", roc_auc_score(y_test, clf_svc.predict_proba(X_test)[:,1]))

In [ ]:
titles = ['RF','LR','SVC','GBT','KNN']
# fitted_models = [clf_rf, gs_logmodel, clf_svc, gs_gb, gs_knn]
estimators = {"RF": clf_RandomForest,
              "LR": gs_logmodel,
              "SVC": clf_svc,
              "GBT": gs_gb,
              "KNN": gs_knn}

print("The accuracy rate and f1-score on test data are:")
for estimator in estimators.keys():
    print("{}: {:.2f}%, {:.2f}%.".format(estimator,
            accuracy_score(y_test, estimators[estimator].predict(X_test)) * 100,
            f1_score(y_test, estimators[estimator].predict(X_test)) * 100))
        
    fpr, tpr, thresholds = roc_curve(y_test, estimators[estimator].predict_proba(X_test)[:,1])
    
    fig, ax = plt.subplots()
    plt.figure(dpi=600)
    
    ax.plot(fpr, tpr,lw=2)
    ax.plot([0,1],[0,1],c='violet',ls='--')
    ax.set_xlim([-0.05,1.05])
    ax.set_ylim([-0.05,1.05])
    
    ax.set_xlabel('False Positive Rate', fontsize=16)
    ax.set_ylabel('True Positive Rate', fontsize=16)
    ax.set_title('Fitted Models ROC curve for IBM Employee Retention',fontsize=16);

    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
              ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(16)

    ax.yaxis.set_label_coords(-0.1,0.5)
    ax.xaxis.set_label_coords(0.5,-0.1)

# Check Balance of Classes
- Look at proportion of classes to see if we're dealing with balanced or imbalanced data, since each one has its own set of tools to be used when fitting classifiers

# Get number of Yes/No Attritions

In [ ]:
# Refit RF classifier
clf_rf = RandomForestClassifier(n_estimators=100,
                                criterion="gini",
                                max_features='log2',
                                min_samples_leaf=1,
                                class_weight="balanced",
                                n_jobs=-1,
                                random_state=123)


clf_rf.fit(StandardScaler().fit_transform(X_smoted), y_smoted)

In [ ]:
#Import Dataframe
with open('./data/ibm_df.pickle', 'rb') as file:
        ibm_df = pickle.load(file)

In [ ]:
ibm_df.shape

In [ ]:
importances

In [ ]:
# Plot features importance
importances = clf_rf.feature_importances_
indices = np.argsort(clf_rf.feature_importances_)[::-1]
plt.figure(figsize=(12, 10))
plt.barh(range(1,41), importances[indices], align="center")
plt.yticks(range(1,41), ibm_df.columns[ibm_df.columns != "Attrition"][indices], rotation=0)
plt.xlabel('Genre', fontsize=5)
plt.title("IBM Employee Attrition Feature Importance", {"fontsize": 18});

In [ ]:
df_columns = list(ibm_df.columns.values)

In [ ]:
df_columns_list = [ ]
for idx, value in enumerate(df_columns):
    df_columns_list.append([value,idx])    

In [ ]:
df_columns_list